# PCE Calculations of Testing Dataset

In [3]:
import ASCVD_Calc_PCE
import pandas as pd
import numpy as np
from importlib import reload
reload(ASCVD_Calc_PCE)
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, accuracy_score

/Users/excenity/Documents/HSIP/Research/Longitudinal PCE/GitHub/Dynamic-DeepHit LRPP/AUC + Risk Calculations/ASCVD_Calc_PCE.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coefs = coefs.append(coefs_list)


## Import Data

In [4]:
df = pd.read_csv('/Volumes/fsmresfiles/PrevMed/Projects/MESA_RiskPred/LRPP data/LRPP_Select_CrossSectional.csv')
indecies_test = pd.read_csv('/Volumes/fsmresfiles/PrevMed/Projects/MESA_RiskPred/LRPP data/random_index_select.csv')
indecies_test['study_index'] = indecies_test['study_index'].astype('bool')
indecies_test['id'] = indecies_test['id'].astype('int')
indecies_test['round'] = indecies_test['round'].astype('int')

In [5]:
def prediction_auc_PCE(pce_pred_df, true_labels):
    
    auc = roc_auc_score(true_labels, pce_pred_df['risk'].tolist())
    
    return auc 

## PCE Calculation

In [15]:
def pce_training(indecies_test, df):
    
    global pce_pred_df
    
    indecies_test = indecies_test.loc[indecies_test['study_index'] == True, :]
    df = df.loc[df['id'].isin(indecies_test['id']), :]
    
    # get labels and tte
    te_label = np.array(df.loc[:,['id', 'label']].drop_duplicates().label)
    te_time = np.array(df.loc[:, ['id', 'time']].drop_duplicates().time)

    # get risk measurements from PCE data set (Multiple)
    pce_pred_df, pce_df = ASCVD_Calc_PCE.pce_pred_df_tab(df, 8, 10)

    # Get PCE AUC for Each Prediction Year
    print('NULL Risk Values from PCE: ', pce_pred_df.risk.isna().sum())
    auc = prediction_auc_PCE(pce_pred_df, te_label)
    print(auc)
    
    return te_label, pce_pred_df['risk']

In [17]:
current_indecies = indecies_test.loc[indecies_test['round'] == 3,:]
true_label, pred_risk = pce_training(current_indecies, df)
output_df = pd.DataFrame({'true_labels': true_label, 
                          'pred_risk': pred_risk})
output_df.to_csv('/Volumes/fsmresfiles/PrevMed/Projects/MESA_RiskPred/Output/pce_auc.csv', index = False)

NULL Risk Values from PCE:  0
0.7677064930856088


In [8]:
auc_list = pd.DataFrame({'round':[],
                         'auc':[]
                        })

index_max = int(indecies_test['round'].max())

In [9]:
for i in range(index_max):
    current_indecies = indecies_test.loc[indecies_test['round'] == i,:]
    print('round:', i)
    auc = pce_training(current_indecies, df)
    auc_chunk = pd.DataFrame({'round': [i],
                              'auc': [auc]
                             })
    auc_list = pd.concat([auc_list, auc_chunk]) 

round: 0
NULL Risk Values from PCE:  0
0.7744813047240032
round: 1
NULL Risk Values from PCE:  0
0.7836565777552555
round: 2
NULL Risk Values from PCE:  0
0.7815426220631403
round: 3
NULL Risk Values from PCE:  0
0.7677064930856088
round: 4
NULL Risk Values from PCE:  0
0.7796398508723384
round: 5
NULL Risk Values from PCE:  0
0.7877124951637337
round: 6
NULL Risk Values from PCE:  0
0.7731954886926619
round: 7
NULL Risk Values from PCE:  0
0.770970633660586
round: 8
NULL Risk Values from PCE:  0
0.7924703802658031
round: 9
NULL Risk Values from PCE:  0
0.7633670152136546
round: 10
NULL Risk Values from PCE:  0
0.7610910678396099
round: 11
NULL Risk Values from PCE:  0
0.7612069683429439
round: 12
NULL Risk Values from PCE:  0
0.7880357642579126
round: 13
NULL Risk Values from PCE:  0
0.7652213592742858
round: 14
NULL Risk Values from PCE:  0
0.7781687778529564
round: 15
NULL Risk Values from PCE:  0
0.7925437640287895
round: 16
NULL Risk Values from PCE:  0
0.7688714764614161
round: 1

KeyboardInterrupt: 

In [10]:
mean = auc_list.auc.mean()
sd = auc_list.auc.std()

auc_lw = round(mean - 1.96*sd/np.sqrt(len(auc_list)),4)
auc_up = round(mean + 1.96*sd/np.sqrt(len(auc_list)),4)

print('AUC:', round(mean,4) , 'CI:', auc_lw, '-', auc_up)

TypeError: complex() first argument must be a string or a number, not 'tuple'

## Get Risk Categories

#### Create Risk Categories
1. Low: < 5% 
2. Borderline: 5% ~ 7.5% 
3. Intermediate: 7.5% ~ 20%
4. High: > 20%

In [ ]:
pce_pred_df['risk_cat'] = 'Low Risk'
pce_pred_df.loc[(pce_pred_df['risk'] >= 0.05) & (pce_pred_df['risk'] < 0.075), 'risk_cat'] = 'Borderline Risk'
pce_pred_df.loc[(pce_pred_df['risk'] >= 0.075) & (pce_pred_df['risk'] < 0.2), 'risk_cat'] = 'Intermediate Risk'
pce_pred_df.loc[(pce_pred_df['risk'] >= 0.2), 'risk_cat'] = 'High Risk'

#### DDH Risk Breakdown by PCE Standards

In [ ]:
round(pce_pred_df.risk_cat.value_counts() / len(pce_pred_df),3)*100

In [ ]:
test = pd.DataFrame(pce_pred_df.groupby('risk_cat').true_label.value_counts()).rename(columns = {'true_label' : 'counts'}).reset_index()
test2 = pd.DataFrame(pce_pred_df.risk_cat.value_counts()).reset_index().rename(columns = {'risk_cat' : 'total_counts', 'index' : 'risk_cat'})
test = pd.merge(test, test2)
test['percentage'] = round(test['counts'] / test['total_counts'],3) * 100
test

In [ ]:
pce_pred_df.to_csv('pce_pred_training_df.csv', index = False)





-----------
## Compare Risk Categories

In [ ]:
ddh_pred_df = pd.read_csv('/Users/excenity/Dropbox/HSIP/Research/MESA/Output/DDH_pred_df.csv')

ddh_pred_df = ddh_pred_df.loc[:, ['pt_id', 'value']].rename(columns = {'value' : 'ddh_risk'})

pred_df = pd.merge(ddh_pred_df, pce_pred_df.loc[:,['risk', 'true_label', 'risk_cat']].reset_index().rename(columns = {'index' : 'pt_id'})).rename(columns = {'risk' : 'pce_risk'})

In [ ]:
# Mean PCE Risk
round(pred_df.groupby('risk_cat').pce_risk.mean(),3)

In [ ]:
# Mean DDH Risk
round(pred_df.groupby('risk_cat').ddh_risk.mean(),3)

### PCE AUC by Risk Category

In [ ]:
# get accuracy scores for each risk category
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'Low Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'Low Risk', 'pce_risk']), 3))
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'Borderline Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'Borderline Risk', 'pce_risk']), 3))
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'Intermediate Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'Intermediate Risk', 'pce_risk']),3))
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'High Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'High Risk', 'pce_risk']), 3))

### DDH AUC by Risk Category

In [ ]:
# get accuracy scores for each risk category
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'Low Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'Low Risk', 'ddh_risk']), 3))
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'Borderline Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'Borderline Risk', 'ddh_risk']), 3))
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'Intermediate Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'Intermediate Risk', 'ddh_risk']),3))
print(round(roc_auc_score(pred_df.loc[pred_df['risk_cat'] == 'High Risk', 'true_label'], pred_df.loc[pred_df['risk_cat'] == 'High Risk', 'ddh_risk']), 3))

In [ ]:
RocCurveDisplay.from_predictions()